In [1]:
import requests
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
raw = pd.read_csv("./data__.csv", encoding="UTF-8") 
raw.head(30)


,방이름,가격,방 수,욕실 수,주소,평 수,관리비,주차 여부,방향,층,총 층,건축물 용도,전세/월세,부동산 이름,부동산 전화번호,복층 여부,사용승인일,내부시설,기타시설
0,일반원룸고층,300/29,1,1,서울시 강북구 우이동 61-19,6.35평/6.35평(전용률100%),5만원,불가능,남향(안방 기준),고,4,단독주택,월세,토마토부동산공인중개사사무소,"02-905-1356,010-7518-1356",단층,2011.11,"침대, 책상, 옷장, 냉장고, 세탁기, 샤워부스, 싱크대, 가스레인지, 인덕션레인지...","현관보안,"
1,일반원룸고층,300/29,2,1,서울시 강북구 우이동 23-38,14.66평/13.11평(전용률89%),0원,불가능,서향(안방 기준),B1,3,공동주택,월세,코끼리공인중개사사무소,"02-992-8989,010-7688-0754",none,1996.05,"사설경비, 베란다,",none
2,빌라B1층,"1,000/45",1,1,서울시 강북구 우이동 71-21,6.35평/5.74평(전용률90%),5만원,불가능,서향(거실 기준),3,4,제2종 근린생활시설,월세,신하나공인중개사사무소,"02-996-1134,010-6342-4597",단층,2005.02.23,none,none
3,일반원룸3층,"3,000/25",1,-,서울시 강북구 우이동 56-45,7.26평/4.95평(전용률68%),5만원,불가능,동향(거실 기준),2,5,공동주택,월세,신하나공인중개사사무소,010-6342-4597,단층,2012.01.02,"냉장고, 신발장, 싱크대, 옷장, 책상, 벽걸이에어컨,","CCTV, 현관보안,"
4,일반원룸2층,500/38,1,-,서울시 강북구 우이동 72-157,6.95평/6.35평(전용률91%),6만원,불가능,동향(거실 기준),2,4,제2종 근린생활시설,월세,신하나공인중개사사무소,010-6342-4597,단층,2019.10.18,"냉장고, 세탁기, 신발장, 싱크대, 옷장, 인덕션레인지, 책상, 벽걸이에어컨,",none
5,일반원룸2층,500/50,1,1,서울시 강북구 우이동 100-57,23.01평/3.81평(전용률17%),0원,불가능,남향(거실 기준),3,5,단독주택,월세,대우공인중개사사무소,"02-997-1881,02-997-1881",단층,2012.03.30,none,none
6,일반원룸3층,300/30,1,1,서울시 강북구 우이동 100-57,23.01평/3.81평(전용률17%),0원,불가능,남동향(거실 기준),고,5,단독주택,월세,대우공인중개사사무소,"02-997-1881,02-997-1881",단층,2012.03.30,"가스레인지, 냉장고, 신발장, 싱크대, 전자레인지, 침대, 벽걸이에어컨,",none
7,일반원룸고층,300/30,1,1,서울시 강북구 우이동 100-28,23.52평/4.53평(전용률19%),0원,가능,동향(거실 기준),중,3,단독주택,월세,토마토부동산공인중개사사무소,"02-905-1356,010-7518-1356",단층,2009,"침대, 책상, 옷장, 냉장고, 세탁기, 인덕션레인지, 벽걸이에어컨,","비디오폰, 현관보안,"
8,일반원룸중층,500/30,1,1,서울시 강북구 우이동 72-66,5.74평/5.14평(전용률89%),0원,불가능,동향(거실 기준),3,3,단독주택,월세,수유역공인중개사사무소,"02-996-7900,010-5250-8779",단층,2016.10.06,"냉장고, 세탁기, 신발장, 싱크대, 인덕션레인지, 벽걸이에어컨,","인터폰,"
9,일반원룸,300/45,1,1,서울시 강북구 우이동 72-66,5.29평/5.14평(전용률97%),0원,불가능,동향(거실 기준),2,3,단독주택,월세,수유역공인중개사사무소,"02-996-7900,010-5250-8779",단층,2016.10.06,"냉장고, 세탁기, 신발장, 싱크대, 인덕션레인지, 벽걸이에어컨,","인터폰,"


In [2]:
rest_api_key = '97527267926acc1dce473f884f0ec30a'

In [3]:
# 대형마트, 편의점, 지하철역, 학교, 문화시설, 음식점, 카페, 약국

places = [['MT1', '1000'], ['CS2', '200'], ['SW8', '500'], ['SC4', '500'], ['CT1', '800'], ['FD6', '300'], ['CE7', '300'], ['PM9', '600']]

In [4]:
def transform_address(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":f"KakaoAK {rest_api_key}"})
    json_obj = result.json()
    document = json_obj['documents'][0]
    val = [document['x'], document['y']]
    return val

In [5]:
# 확인
print(transform_address('서울시 강남구 도곡동 546-4'))

['127.037847689739', '37.491234768653']


In [6]:
def category_count(places, x, y):
    val = []
    for place in places:
        url = "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=" + place[0] + "&x=" + x + "&y=" + y + "&radius=" + place[1]
        result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":f"KakaoAK {rest_api_key}"})
        json_obj = result.json()
        meta = json_obj['meta']
        val.append(meta['total_count'])
    return val

In [8]:
# 확인
location = transform_address('서울시 동작구 상도동 487-11')
x, y = location[0], location[1]
print(category_count(places, x, y))
# 대형마트, 편의점, 지하철역, 학교, 문화시설, 음식점, 카페, 약국

[2, 8, 1, 2, 2, 124, 24, 11]
